In [15]:
import pandas as pd
autentica = pd.read_excel('ls.xlsx')

# display(autentica)


login = str(autentica['login'][13])
senha = str(autentica['senha'][13])

indisponivel = "indisponivel"

In [16]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import Select
import time

def abrir_suap():
    
    global nav
    nav = webdriver.Chrome()
    nav.get("https://suap.ifpe.edu.br/accounts/login/?next=/")
    # inserindo usuário
    nav.find_element_by_xpath('//*[@id="id_username"]').click()
    nav.find_element_by_xpath('//*[@id="id_username"]').send_keys(login)

    # inserindo senha 
    nav.find_element_by_xpath('//*[@id="id_password"]').click()
    nav.find_element_by_xpath('//*[@id="id_password"]').send_keys(senha)

    nav.find_element_by_xpath('//*[@id="ok"]').click()

In [17]:
df_elemento_despesa = pd.DataFrame()
df_temp = pd.DataFrame(columns=('value', 'ed'))
df_temp.loc[0] = ['14', 'esportes']
# display(df_temp)

opcoes = []
valores = []

def varrer_ed():
#     abrir_suap()
    nav.get("https://suap.ifpe.edu.br/almoxarifado/form_busca/materialconsumo/")
        
    valor_ed = nav.find_element_by_id('id_elemento_despesa')
    options = [x for x in valor_ed.find_elements_by_tag_name('option')]
    
    for i in options:
        # print(i.get_attribute('value')+" "+i.text)
        opcoes.append(i.text)
        valores.append(i.get_attribute('value'))
#         print(f'{i.text}-{i.get_attribute("value")}')

    df_elemento_despesa['value'] = valores
    df_elemento_despesa['ed']= opcoes
    
#     display(df_elemento_despesa)

In [18]:
def get_qtd():
    
    abrir_suap()
    
    df_tabela = pd.DataFrame()
    tabela = []
    codigos = []
    descricoes = []
    quantidades = []
    valores = []
    elemento = []
    
    try:
        pag = 1
        saida = True
        while saida:
                
            nav.get(f'https://suap.ifpe.edu.br/almoxarifado/form_busca/materialconsumo/?page={pag}&material=&elemento_despesa=&estoque_tipo=emcampus&unidade_medida=')
            celula = nav.find_element_by_tag_name('td')
                
            if celula:
                lista = nav.find_element_by_tag_name('table')
                td = lista.find_elements_by_tag_name('td')
                
                for i in td:
                    tabela.append(i.text)
                
                pag += 1
            else:
                saida = False
                
    except NoSuchElementException:
        print('acabaram as páginas')
        
    for cod in range(0, len(tabela), 7):
        codigos.append(tabela[cod])

    for des in range(1, len(tabela), 7):
        descricoes.append(tabela[des])

    for qtd in range(4, len(tabela), 7):
        quantidades.append(tabela[qtd])
    
    for val in range(5, len(tabela), 7):
        valores.append(tabela[val])
        
    for ed in range(3, len(tabela), 7):
        elemento.append(tabela[ed])

    df_tabela['codigo'] = codigos
    df_tabela['descricao'] = descricoes
    
    df_tabela['qtd'] = quantidades
    df_tabela['qtd'] = pd.to_numeric(df_tabela['qtd'])
    
    df_tabela['vlr_total'] = valores
    df_tabela['vlr_total'] = df_tabela['vlr_total'].str[2:]
    df_tabela['vlr_total'] = pd.to_numeric(df_tabela['vlr_total'])
    
    df_tabela['vlr_unt'] = df_tabela['vlr_total'] / df_tabela['qtd']
    df_tabela['vlr_unt'] = df_tabela['vlr_unt'].round(2)
    
    df_tabela['ed'] = elemento
    
    df_tabela.to_excel('relatorio/relatorio.xlsx', index=False)
    display(df_tabela)
    
    nav.quit()


In [19]:
get_qtd()

acabaram as páginas


,codigo,descricao,qtd,vlr_total,vlr_unt,ed
0,30.07.862,30.07.351 ÁGUA MINERAL EM GARRAFÃO DE 20 LITRO...,250,800.00,3.20,07 - GENEROS DE ALIMENTACAO
1,30.11.922,"Álcool 70. agente desinfetante, Líquido, límpi...",89,4628.00,52.00,11 - MATERIAL QUIMICO
2,30.16.1054,"CANETA DESENHO ARQUITETONICO, MATERIAL CORPO P...",4,4.92,1.23,16 - MATERIAL DE EXPEDIENTE
3,30.16.1065,"ESTOJO PARA DVD, EM PVC 19 X 13,5 X 1,3CM - TR...",1,2.08,2.08,16 - MATERIAL DE EXPEDIENTE
4,30.16.1076,"PAPEL CARTOLINA, MATERIAL CELULOSE VEGETAL, GR...",10,3.50,0.35,16 - MATERIAL DE EXPEDIENTE
...,...,...,...,...,...,...
340,30.42.233,"ENXADA 1,5 LIBRAS",2,55.96,27.98,42 - FERRAMENTAS
341,30.42.269,"FORCADO, MATERIAL AÇO SAE 1045/60, 4 DENTES.",2,100.08,50.04,42 - FERRAMENTAS
342,30.42.34,MACHADO P/ CORTAR MADEIRA (C/ CABO),2,77.98,38.99,42 - FERRAMENTAS
343,30.42.959,"ALICATE REBITADOR, MANUAL, CABO EMBORRACHADO, ...",4,64.00,16.00,42 - FERRAMENTAS
